# Car Prediction #
İkinci el araç fiyatlarını (özelliklerine göre) tahmin eden modeller oluşturma ve MLOPs ile Hugging Face üzerinden yayımlayacağız.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split #veri setini bölme işlemleri
from sklearn.linear_model import LinearRegression #Doğrusal regresyon
from sklearn.metrics import r2_score,mean_squared_error #modelimizin performansını ölçmek için
from sklearn.compose import ColumnTransformer #Sütun dönüşüm işlemleri
from sklearn.preprocessing import OneHotEncoder, StandardScaler # kategori - sayısal dönüşüm  ve ölçeklendirme
from sklearn.pipeline import Pipeline #Veri işleme hattı

In [2]:
#Excell dosyalarını okumak için

In [7]:
%pip install xlrd


   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ------------ --------------------------- 30.7/96.5 kB 217.9 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/96.5 kB 196.9 kB/s eta 0:00:01
   -------------------------------------- - 92.2/96.5 kB 374.1 kB/s eta 0:00:01
   -------------------------------------- - 92.2/96.5 kB 374.1 kB/s eta 0:00:01
   -------------------------------------- - 92.2/96.5 kB 374.1 kB/s eta 0:00:01
   ---------------------------------------- 96.5/96.5 kB 263.1 kB/s eta 0:00:00


## Veri dosyasını yükle

In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is BA88-10BB

 Directory of C:\Users\aykut\OneDrive\Masa�st�\Techist\ders10

17.06.2024  16:14    <DIR>          .
17.06.2024  16:14    <DIR>          ..
17.06.2024  16:11    <DIR>          .ipynb_checkpoints
17.06.2024  16:10             2.555 app.py
17.06.2024  16:13            51.405 car_predict_tech.ipynb
17.06.2024  16:14           141.824 cars.xls
17.06.2024  16:14             3.399 Untitled.ipynb
               4 File(s)        199.183 bytes
               3 Dir(s)  166.932.238.336 bytes free


In [8]:
df=pd.read_excel('cars.xls')
df

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
799,16507.070267,16229,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,0,0
800,16175.957604,19095,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0
801,15731.132897,20484,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0
802,15118.893228,25979,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0


df.info()

In [10]:
# Veri ön işleme

In [11]:
X=df.drop('Price',axis=1) #fiyat sütunu çıkar fiyata etki edenler kalsın
y=df['Price'] #tahmin

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#### Veri ön işleme, standartlaştırma ve OHE işlemlerini otomatikleştiriyoruz (standartlaştırıyoruz). Artık preprocess kullanarak kullanıcında arayüz aracılığıyla gelen veriyi mdoelimize uygun hale çevirebiliriz.

In [13]:
preprocess=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),['Mileage', 'Cylinder','Liter','Doors']),
        ('cat',OneHotEncoder(),['Make','Model','Trim','Type'])
    ]
)

In [14]:
my_model=LinearRegression()

In [15]:
#pipeline ı tanımla
pipe=Pipeline(steps=[('preprocessor',preprocess),('model',my_model)])

In [16]:
#pipeline fit
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Mileage', 'Cylinder',
                                                   'Liter', 'Doors']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Make', 'Model', 'Trim',
                                                   'Type'])])),
                ('model', LinearRegression())])

In [17]:
y_pred=pipe.predict(X_test)
print('RMSE',mean_squared_error(y_test,y_pred)**0.5)
print('R2',r2_score(y_test,y_pred))

RMSE 835.1013862206736
R2 0.9912072687898575


In [18]:
#isterseniz veri setinin tammamıyla tekrar eğitim yapabilirsiniz.
#pipe.fit(X,y)

## Streamlit ile modeli yayma/deploy/kullanıma sunma

In [19]:
!pip install streamlit

   ---------------------------------------- 0.0/8.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.6 MB 3.4 MB/s eta 0:00:03
    --------------------------------------- 0.1/8.6 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.2/8.6 MB 1.6 MB/s eta 0:00:06
   - -------------------------------------- 0.4/8.6 MB 1.9 MB/s eta 0:00:05
   - -------------------------------------- 0.4/8.6 MB 2.0 MB/s eta 0:00:05
   - -------------------------------------- 0.4/8.6 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.4/8.6 MB 1.5 MB/s eta 0:00:06
   -- ------------------------------------- 0.4/8.6 MB 1.5 MB/s eta 0:00:06
   ---- ----------------------------------- 0.9/8.6 MB 2.1 MB/s eta 0:00:04
   ------- -------------------------------- 1.5/8.6 MB 3.2 MB/s eta 0:00:03
   -------- ------------------------------- 1.7/8.6 MB 3.3 MB/s eta 0:00:03
   -------- ------------------------------- 1.9/8.6 MB 3.4 MB/s eta 0:00:02
   --------- ------

In [20]:
df['Mileage'].max()

50387

In [21]:
df['Type'].unique()

array(['Sedan', 'Convertible', 'Hatchback', 'Coupe', 'Wagon'],
      dtype=object)

In [22]:
df['Liter'].max()

6.0

#### Python ile yapılan çalışmaların hızlı bir şekilde deploy edilmesi için HTML render arayüzler tasarlamanızı sağlar.

In [26]:
import streamlit as st
#price tahmin fonksiyonu tanımla
def price(make,model,trim,mileage,car_type,cylinder,liter,doors,cruise,sound,leather):
    input_data=pd.DataFrame({'Make':[make],
                             'Model':[model],
                             'Trim':[trim],
                             'Mileage':[mileage],
                             'Type':[car_type],
                             'Cylinder':[cylinder],
                             'Liter':[liter],
                             'Doors':[doors],
                             'Cruise':[cruise],
                             'Sound':[sound],
                             'Leather':[leather]})
    prediction=pipe.predict(input_data)[0]
    return prediction
st.title("II. El Araba Fiyatı Tahmin:red_car: @aykutsahin")
st.write('Arabanın özelliklerini seçiniz')
make=st.selectbox('Marka',df['Make'].unique())
model=st.selectbox('Model',df[df['Make']==make]['Model'].unique())
trim=st.selectbox('Trim',df[(df['Make']==make) &(df['Model']==model)]['Trim'].unique())
mileage=st.number_input('Kilometre',100,200000)
car_type=st.selectbox('Araç Tipi',df[(df['Make']==make) &(df['Model']==model)&(df['Trim']==trim)]['Type'].unique())
cylinder=st.selectbox('Cylinder',df['Cylinder'].unique())
liter=st.number_input('Yakıt hacmi',1,10)
doors=st.selectbox('Kapı sayısı',df['Doors'].unique())
cruise=st.radio('Hız Sbt.',[True,False])
sound=st.radio('Ses Sis.',[True,False])
leather=st.radio('Deri döşeme.',[True,False])
if st.button('Tahmin'):
    pred=price(make,model,trim,mileage,car_type,cylinder,liter,doors,cruise,sound,leather)
    st.write('Fiyat:$', round(pred[0],2))

In [24]:
#streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py